# Tonga 5m DEM creation
Load in LiDAR where it exists and combine with FABDEM to produce 5m DEMs

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pathlib

import os
import sys
module_path = os.path.abspath(os.path.join("..", "py_scripts"))
if module_path not in sys.path:
    sys.path.append(module_path)

import create_dem_functions
import islands
import create_data_paths

# Define country and resolution

In [ ]:
resolution = 5 # in meters
country_name = "tuvalu"

# Create paths and get contry information

In [ ]:
paths = create_data_paths.get_paths(country_name=country_name, resolution=resolution)
islands = islands.get_island(country_name)

# Specify any LiDAR data

In [ ]:
# Path of LiDAR DEMs
vaitupu_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_vaiupu.nc", masked=True)
nukufetau_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_nukufetau.nc", masked=True)
nanumanga_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_nanumanga.nc", masked=True)
nanumea_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_nanumea.nc", masked=True)
niulakita_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_niulakita.nc", masked=True)
niutao_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_niutao.nc", masked=True)
nui_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_nui.nc", masked=True)
nukulaelae_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_nukulaelae.nc", masked=True)
funafuti_lidar = rioxarray.open_rasterio(paths["lidar"] / "1m_dem_tv_funafuti.nc", masked=True)

# Add to islands
island_dict["vaitupu"]["lidar"] = vaitupu_lidar
island_dict["nukufetau"]["lidar"] = nukufetau_lidar
island_dict["nanumanga"]["lidar"] = nanumanga_lidar
island_dict["nanumea"]["lidar"] = nanumea_lidar
island_dict["niulakita"]["lidar"] = niulakita_lidar
island_dict["niutao"]["lidar"] = niutao_lidar
island_dict["nui"]["lidar"] = nui_lidar
island_dict["nukulaelae"]["lidar"] = nukulaelae_lidar
island_dict["funafuti"]["lidar"] = funafuti_lidar

# Get island outlines and combine FABDEMs

In [ ]:
for island_name, island_dict in islands.items():
    print(f"Setup for {island_name}")
    island_dict["name"] = island_name
    island_dict["boundary"] = create_dem_functions.create_boundary_epsg4326(island_dict=island_dict, output_path=paths["output"])
    island_dict["fab"] = create_dem_functions.combine_fabs_in_boundary(island_dict=island_dict, fab_path=paths["fabdem"])

# Create DEMs over islands

In [ ]:
create_dem_functions.loop_through_islands_creating_dems(island_groups=islands,
                                                        resolution=resolution, output_path=paths["output"])